# Zaawansowane Przetwarzanie Języka Naturalnego
## Laboratorium 3: Neuronowe modele predykcji sekwencji

Celem zadania jest zaimplementowanie neuronowego modelu do predykcji sekwencji i zastosowanie go do zadania płytkiej analizy frazowej. Zaimplementowany model będzie oparty o dwukierunkowe sieci rekurencyjne oraz zakończony będzie warstwą CRF. Celem ćwiczenia nie jest stworzenie wydajnej implementacji odpowiedniej do pracy z dużymi zbiorami treningowymi, ale pogłębienie zrozumienia działania metod modelowania predykcji poprzez ich implementację.

## Zadanie 1
Pracując w środowisku PyTorch nie jest konieczne samodzielne implementowanie neuronów rekurencyjnych, gdyż biblioteka `torch.nn` zawiera ich gotowe implementacje. Klasyczny neuron rekurencyjny z funkcją aktywacji tangens hiperboliczny jest zaimplementowany w klasie `RNN` i od razu pozwala na przetworzenie od razu całej sekwencji elementów. Konstruktor obiektu wymaga wyspecyfikowania liczby cech którymi opisany jest każdy element sekwencji, liczby neuronów w warstwie rekurenycjnej oraz liczbę warstw rekurencyjnych. 

Istotnym parametrem modelu jest opcjonalny przełącznik `batch_first = True` który określa format wejściowej sekwencji. Domyślnie modele rekurencyjne spodziewają się wejścia o wymiarach `(długość sekwencji, rozmiar batcha, liczba cech)` czyli w kolumnach tensora mamy kolejne sekwencje (przykłady uczące), w wierszach kolejne elementy tych sekwencji, a w głębokości tensora umieszone są kolejne cechy każdego z elementów sekwencji. Zwróć uwagę, że jest to format inny od często stosowanego formatu danych niesekwencyjnych gdzie w wierszach umieszcza się kolejne przykłady uczące, a w kolumnach kolejne cechy (tutaj jest na odwrót: przykład uczący-sekwencja umieszcona jest w kolumnie, a nie w wierszu). Jeśli jednak ustawisz przełącznik `batch_first = True` to sieć rekurencyjna będzie się spodziewała wejścia jako `(rozmiar batcha, długość sekwencji, liczba cech)` czyli kolejne sekwencje będą w kolejnych wierszach tensora.

Dalej, warto zauważyć, że w przypadku np. warstwy liniowej do obliczenia wyniku wystarczyło podanie samych danych ( `linear(dane)`), jednak wejściem sieci rekurencyjnej są nie tylko dane ale i poprzednie stany ukryte $h_{t-1}$ ( `rnn(dane, h0)`).

In [2]:
import torch
import torch.nn as nn

N_FEATURES = 2
N_LAYERS = 1
HIDDEN_SIZE = 3

data = torch.FloatTensor([[1,1], [2,2], [3,3], [4,4]]).view(1, 4, 2)
# Proste przykładowe dane: jedna 4 elementowa sekwencja gdzie każdy jej element jest opisany dwoma cechami
# Zwróć uwagę na wymiary tensora (rozmiar batcha, długość sekwencji, liczba cech elementów) 
#     - konieczne batch_first=True

rnn = nn.RNN(N_FEATURES, HIDDEN_SIZE, N_LAYERS, batch_first = True) 

h0 = torch.randn(N_LAYERS, 1, HIDDEN_SIZE)
# Początkowy stan ukryty h0. Każda warstwa sieci rekurencyjnej zaczyna przetwarzanie od swojego h0
# W sytuacji gdy przetwarzamy kilka sekwencji na raz (sieć jest uruchamiana równolegle na kilku sekwencjach)
# również potrzebujemy dla każdej przetwarzanej sekwencji jej stan początkowy h0
# Porównaj powyższy opis z wymiarowością zmiennej h0

out, h_t = rnn(data, h0)
print("Wyjście warstwy: ", out)
print("Ostatni stan ukryty: ",h_t)


Wyjście warstwy:  tensor([[[ 0.9394,  0.6621, -0.4584],
         [ 0.9115,  0.4247, -0.6293],
         [ 0.9285,  0.2709, -0.7568],
         [ 0.9482,  0.1473, -0.8483]]], grad_fn=<TransposeBackward1>)
Ostatni stan ukryty:  tensor([[[ 0.9482,  0.1473, -0.8483]]], grad_fn=<StackBackward0>)


Zwróć uwagę na otrzymane wyjście: ponieważ na wejście sieci podaliśmy sekwencję elementów to dla każdego przetwarzanego elementu sekwencji sieć zwróciła uzyskaną reprezentację ukrytą (skoro mamy 3 neurony w warstwie to taka reprezentacja jest 3-wymiarowa). Dodatkowo zwrócony został ostatni stan ukryty, który moglibyśmy przetwarzać np. kolejnej sieci rekurencyjnej.

Sprawdź poprawność uzyskanego powyżej wyniku poprzez uruchomienie sieci rekurencyjnej element po elemencie (wejściem do sieci jest zawsze tylko jeden element sekwencji lub inaczej: sekwencje 1-elementowe). Wypisz na wyjście kolejne otrzymywane wyniki sieci i jej stany ukryte. Uzyskane wyniki powinny być takie same jak te z komórki wyżej.

In [3]:
h1 = h0
for i in range(1,5):
    x_i = torch.FloatTensor([[i,i]]).view(1,1,2)
    out, h1 = rnn(x_i, h1)
    print(f"Wyjście warstwy {i}: ", out)
    print(f"Ostani stan ukryty {i}: ", h1)

Wyjście warstwy 1:  tensor([[[ 0.9394,  0.6621, -0.4584]]], grad_fn=<TransposeBackward1>)
Ostani stan ukryty 1:  tensor([[[ 0.9394,  0.6621, -0.4584]]], grad_fn=<StackBackward0>)
Wyjście warstwy 2:  tensor([[[ 0.9115,  0.4247, -0.6293]]], grad_fn=<TransposeBackward1>)
Ostani stan ukryty 2:  tensor([[[ 0.9115,  0.4247, -0.6293]]], grad_fn=<StackBackward0>)
Wyjście warstwy 3:  tensor([[[ 0.9285,  0.2709, -0.7568]]], grad_fn=<TransposeBackward1>)
Ostani stan ukryty 3:  tensor([[[ 0.9285,  0.2709, -0.7568]]], grad_fn=<StackBackward0>)
Wyjście warstwy 4:  tensor([[[ 0.9482,  0.1473, -0.8483]]], grad_fn=<TransposeBackward1>)
Ostani stan ukryty 4:  tensor([[[ 0.9482,  0.1473, -0.8483]]], grad_fn=<StackBackward0>)


W prosty sposób można również rozszerzyć naszą warstwę rekurencyjną do sieci dwukierunkowej. W konstruktorze wystarczy ustawić `bidirectional = True`. Jak mówiliśmy na wykładzie taka sieć składa się w rzeczywistości z dwóch sieci - jednej przetwarzającej wyniki od lewej do prawej i drugiej przetwarzającej wyniki od prawej do lewej. W związku z tym należy dla obydwu tych sieci przygotować ich stany początkowe $h_0$, a wymiarowość wyjścia zwiększy się dwukrotnie.

In [4]:
rnn = nn.RNN(N_FEATURES, HIDDEN_SIZE, N_LAYERS, batch_first = True, bidirectional = True) 

h0 = torch.randn(N_LAYERS*2, 1, HIDDEN_SIZE)
out, h_t = rnn(data, h0)
print("Wyjście warstwy: ", out)
print("Ostatni stan ukryty: ",h_t)


Wyjście warstwy:  tensor([[[ 0.5627, -0.7731, -0.7677,  0.1548,  0.0299, -0.8522],
         [ 0.7930, -0.8384, -0.8800, -0.0808,  0.3933, -0.9566],
         [ 0.9137, -0.9432, -0.9126,  0.0591,  0.4522, -0.9852],
         [ 0.9672, -0.9785, -0.9420, -0.8571,  0.9156, -0.9967]]],
       grad_fn=<TransposeBackward1>)
Ostatni stan ukryty:  tensor([[[ 0.9672, -0.9785, -0.9420]],

        [[ 0.1548,  0.0299, -0.8522]]], grad_fn=<StackBackward0>)


Analogicznie jak poprzednio uzyskaj wynik z poprzedniej komórki zakładając że na wejściu obiektu `rnn` możesz umieścić jedynie sekwencje jednoelementowe (możesz go jednak wywoływać dowolnie wiele razy).

In [5]:
h1 = h0
for i in range(1, 5):
    x_i = torch.FloatTensor([[i,i]]).view(1,1,2)
    out, h1 = rnn(x_i, h1)
    print(f"Wyjście warstwy {i}: ", out)
    print(f"Ostatni stan ukryty {i}: ",h_t)

Wyjście warstwy 1:  tensor([[[ 0.5627, -0.7731, -0.7677, -0.6480,  0.4740, -0.8734]]],
       grad_fn=<TransposeBackward1>)
Ostatni stan ukryty 1:  tensor([[[ 0.9672, -0.9785, -0.9420]],

        [[ 0.1548,  0.0299, -0.8522]]], grad_fn=<StackBackward0>)
Wyjście warstwy 2:  tensor([[[ 0.7930, -0.8384, -0.8800,  0.1954,  0.1981, -0.9479]]],
       grad_fn=<TransposeBackward1>)
Ostatni stan ukryty 2:  tensor([[[ 0.9672, -0.9785, -0.9420]],

        [[ 0.1548,  0.0299, -0.8522]]], grad_fn=<StackBackward0>)
Wyjście warstwy 3:  tensor([[[ 0.9137, -0.9432, -0.9126, -0.2611,  0.6726, -0.9872]]],
       grad_fn=<TransposeBackward1>)
Ostatni stan ukryty 3:  tensor([[[ 0.9672, -0.9785, -0.9420]],

        [[ 0.1548,  0.0299, -0.8522]]], grad_fn=<StackBackward0>)
Wyjście warstwy 4:  tensor([[[ 0.9672, -0.9785, -0.9420, -0.3147,  0.7651, -0.9961]]],
       grad_fn=<TransposeBackward1>)
Ostatni stan ukryty 4:  tensor([[[ 0.9672, -0.9785, -0.9420]],

        [[ 0.1548,  0.0299, -0.8522]]], grad_fn=<S

Na koniec warto też zauważyć, że samodzielna inicjalizacja stanu wejściowego do sieci rekurencyjnej `h0` nie jest obowiązkowa i na wejście warstwy rekurencyjnej można podać samą sekwencję wejściową. W takim wypadku element `h0` zostanie zainicjalizowany domyślnie wektorem zer.

In [6]:
out, h_t = rnn(data)
print("Wyjście warstwy: ", out)
print("Ostatni stan ukryty: ",h_t)


Wyjście warstwy:  tensor([[[ 0.6302, -0.7406, -0.7995,  0.1490,  0.0401, -0.8527],
         [ 0.7869, -0.8504, -0.8728, -0.0532,  0.3630, -0.9559],
         [ 0.9139, -0.9424, -0.9135, -0.0443,  0.5381, -0.9860],
         [ 0.9672, -0.9785, -0.9420, -0.5081,  0.6718, -0.9937]]],
       grad_fn=<TransposeBackward1>)
Ostatni stan ukryty:  tensor([[[ 0.9672, -0.9785, -0.9420]],

        [[ 0.1490,  0.0401, -0.8527]]], grad_fn=<StackBackward0>)


**Ćwiczenia**
- Jakich zmian w kodzie musiałbyś dokonać by uzyskać 3-warstwową sieć rekurencyjną? Jak zmieniłaby się wtedy wymiarowośc `h0` oraz wyjścia sieci neuronowej `h1` i `out`?
- W przypadku neuronu rekurencyjnego RNN wejściem do modelu jest sekwencja oraz początkowy/poprzedni stan ukryty `h0`. Korzystając z gotowej implemetancji neuronu LSTM jakiego wejścia się spodziewasz?

Odpowiedzi nie musisz zapisywać.

## Zadanie 2
W ostatnim zadaniu domowym poznawałeś usprawnienia implementacji modeli uczących się w PyTorch poprzez wykorzystanie gotowych elementów z modułu `torch.nn`. Jednak ostatecznie uzyskany przez nas kod nadal przetwarzał pojedyncze instancje tj. nieobsługiwał mini-batchy. Można oczywiście ręcznie zaimplementować indeksowanie, które pozwoli nam na iterowanie po paczkach danych, jednak PyTorch pozwala oczywiście na automatyzację tego procesu.

Na początek wczytajmy dane uczące z których będziemy korzystać w tym zadaniu. (Może być konieczna instalacja biblioteki `torchtext`).

In [7]:
from torchtext.datasets import CoNLL2000Chunking
train_iter = CoNLL2000Chunking(split='train')
seq_texts = []
seq_tags = []
for i in train_iter:
    seq_texts.append(i[0])
    seq_tags.append(i[2])
print(seq_texts[0])
print(seq_tags[0])

['Confidence', 'in', 'the', 'pound', 'is', 'widely', 'expected', 'to', 'take', 'another', 'sharp', 'dive', 'if', 'trade', 'figures', 'for', 'September', ',', 'due', 'for', 'release', 'tomorrow', ',', 'fail', 'to', 'show', 'a', 'substantial', 'improvement', 'from', 'July', 'and', 'August', "'s", 'near-record', 'deficits', '.']
['B-NP', 'B-PP', 'B-NP', 'I-NP', 'B-VP', 'I-VP', 'I-VP', 'I-VP', 'I-VP', 'B-NP', 'I-NP', 'I-NP', 'B-SBAR', 'B-NP', 'I-NP', 'B-PP', 'B-NP', 'O', 'B-ADJP', 'B-PP', 'B-NP', 'B-NP', 'O', 'B-VP', 'I-VP', 'I-VP', 'B-NP', 'I-NP', 'I-NP', 'B-PP', 'B-NP', 'I-NP', 'I-NP', 'B-NP', 'I-NP', 'I-NP', 'O']


Są to dane z popularnego zbioru CoNLL2000 dotyczące zadania płytkiej analizy frazowej. W liście `seq_texts` umieszczono kolejne zdania, a na liście `seq_tags` umieszczono odpowiadające im zestawy tagów. Zwróć uwagę, że zastosowano tutaj tagowanie BIO np. frazą czasownikową `VP` jest "is widely expected to take".

Pierwszym krokiem przetwarzania jest zamiana słów na ich identyfikatory oraz obsłużenie tokenu `OOV`. W poprzednich zadaniach domowych robiliśmy to ręcznie jednak dzięki bibliotece `torchtext` ten proces równiez możemy zautomatyzować dzięki obiektowi `Vocab` i funkcji `build_vocab_from_iterator`.

In [64]:
from torchtext.vocab import build_vocab_from_iterator
vocab = build_vocab_from_iterator(seq_texts, specials=["<unk>"], min_freq=5)
len(vocab)

4403

Funkcja ta na wejście przyjmuje iterator z tekstami, a także opcjonalnie próg `min_freq` pomijający słowa występujące z niewystarczającą częstotliowścią oraz `specials` pozwalający na umieszczenie w słowniku dodatkowych tokenów. Tokeny specjalne domyślnie są umieszczane na początku słownika.

Obiekt `Vocab` w prosty sposób zamienia sekwencję tokenów na sekwencję ich indeksów.

In [8]:
vocab(["if","could", "in", "<unk>"])

[107, 80, 8, 0]

oraz na inne proste konwersje:

In [9]:
print(vocab.lookup_token(107))
print(vocab.lookup_indices(["<unk>"]))

if
[0]


Domyślnie jednak słownik nie obsługuje słów spoza słownika. Dla przykładu "Confidence" występuje w rozważanym korpusie mniej niż 2 razy.

In [10]:
vocab(["Confidence"])

RuntimeError: Token Confidence not found and default index is not set

Można jednak ustawić domyślny indeks słownika na token `UNK`, który automatycznie zamienia nieznane słowa na indeks tego tokenu.

In [11]:
vocab.set_default_index(0)
vocab(["if","could", "in", "<unk>","Confidence"])

[107, 80, 8, 0, 0]

Następny krokiem jest implementacja własnej klasy zbioru danych, dziedziczącej po klasie `torch.utils.data.Dataset`, która zapewni kompatybilność reprezentacji naszych danych z procedurami m.in. automatycznie tworzącymi batche.

Zbiór danych powinien implementować co najmniej 3 funkcje: konstruktor, funkcję zwracającą liczbę elementów w zbiorze `__len__` oraz funckję pozwalającą na dostęp do wybranego elementu zbioru `__getitem__`.

Wykorzystując funkcję `build_vocab_from_iterator` uzupełnij implementację poniższej klasy, tak aby funkcja `__getitem__` zwracała elementy zbioru składające się z sekwencji indeksów słów oraz z sekwencji indeksów klas. Należy obsłużyć słowa spoza słownika zamieniająć na token `UNK` słowa występujące jednokrotnie w zbiorze danych. Dodatkowo słownik powinien uwzględniać token specjalny `<PAD>`, najlepiej umieszczony pod indeksem 0, który będzie potrzebny w dalszej części ćwiczenia.

In [62]:
from torch.utils.data import Dataset, DataLoader

class CoNLLDataset(Dataset):
    def __init__(self, seq_texts, seq_tags):
        self.texts = seq_texts
        self.labels = seq_tags
        self.vocab = build_vocab_from_iterator(seq_texts, specials=["<pad>", "<unk>"], min_freq=2)
        self.vocab.set_default_index(1)
        self.tagset = build_vocab_from_iterator(seq_tags, specials=["<unk>"], min_freq=1)
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, item):
        return {'text':self.vocab(self.texts[item]), 'label':self.tagset(self.labels[item])}
    
    def get_vocab_size(self):
        return len(self.vocab)
    
    def get_tagset_size(self):
        return len(self.tagset)

In [65]:
dataset = CoNLLDataset(seq_texts, seq_tags)
for i in dataset:
    print(i)
    break

{'text': [1, 9, 3, 1775, 17, 1164, 177, 6, 212, 317, 1216, 6116, 108, 650, 597, 11, 457, 2, 249, 11, 2626, 2963, 2, 4920, 6, 602, 7, 1401, 1752, 21, 732, 8, 543, 10, 8902, 1, 4], 'label': [2, 5, 2, 1, 4, 6, 6, 6, 6, 2, 1, 1, 8, 2, 1, 5, 2, 3, 9, 5, 2, 2, 3, 4, 6, 6, 2, 1, 1, 5, 2, 1, 1, 2, 1, 1, 3]}


Mając zaimplementowany zbiór danych jako obiekt typu `Dataset`, podzielenie go na paczki instancji nie jest trudne. Wystarczy wykorzystać obiekt `DataLoader` i wyspecyfikować w nim rozmiar paczki danych `batch_size`.

In [14]:
dataloader = DataLoader(dataset, batch_size=4)
print("Ile utworzono paczek?", len(dataloader))

Ile utworzono paczek? 2234


Niemniej jednak praca z tekstami nie jest niestety taka prosta spróbuj przeiterować pod kolejnych paczkach twojego zbioru:

In [15]:
for i in dataloader:
    print(i)
    break

RuntimeError: each element in list of batch should be of equal size

Obiekt `DataLoader` próbuje automatycznie podzielić nasze dane na paczki, jednak nie jest to takie proste. Nasze dane składają się z sekwencji o różnych długościach, ciężko jest więc utworzyć z nich eleganckie tensory które mają stałe wymiarowości. Naturalnym rozwiązaniem problemu jest wyznaczenie długości najdłuższej sekwencji i uzupełnienie wszystkich pozostałych sekwencji do tej długości specjlanymi tokenami `<PAD>`. Taka operacja może niestety znacznie zwiększyć wymagania pamięciowe potrzebne do reprezentacji zbioru. Pojawienie się jednej bardzo długiej sekwencji w zbiorze znacznie zwiększa czas przetwarzania dla wszystkich jego elementów. 

Zauważ, że wymaganiem technicznym nie jest posiadanie całego zbioru danych w postaci jednego dużego tensora, ale stworzenie takiego tensora dla wszystkich sekwnencji w ramach jednej paczki danych. Zwykle długość najdłużej sekwencji w paczce jest dużo mniejsza niż długość najdłuszej sekwencji w całym zbiorze, co pozwoliłoby nam na nie tylko znaczne lepsze wykorzystanie pamięci operacyjnej, ale także na ograniczenie czasu przetwarzania. Z tego powodu w tym zadaniu będziemy dynamicznie tworzyć reprezentację paczki danych -- każda paczka będzie zawierała tyle samo sekwencji, jednak będą one uzupełniane do różnych długości, pod tym względem paczki danych *nie* będą równe.  

Do uzupełnienia sekwencji do równych długości przydatna będzie funkcja `pad_sequence`, która również posiada parametr `batch_first` przygotowując dane o odpowiednich wymiarach.

In [69]:
from torch.nn.utils.rnn import pad_sequence

data = [torch.tensor([1,2,3]), torch.tensor([1,2])]
pad_data = pad_sequence(data, padding_value=0)
print(pad_data)

pad_data = pad_sequence(data, padding_value=0, batch_first=True)
print(pad_data)

tensor([[1, 1],
        [2, 2],
        [3, 0]])
tensor([[1, 2, 3],
        [1, 2, 0]])


Warto zwrócić uwagę, że zrównoleglanie przetwarzania sieci rekurencyjnej nie jest proste, gdyż wyniki przetwarzania zależą od poprzednich wyników. Nie można więc, tak jak w przypadku sieci splotowej, równocześnie obliczyć wyników filtra na wszystkich słowach w tekście, znakomicie zrównoleglając przetwarzanie. Należy najpierw zastosować "filtr" na pierwszym słowie, poczekać na wynik, potem na kolejnym itd. W związku z tym najlepszą możliwością zrównoleglenia przetwarzania sieci rekurencyjnych jest obsługa wielu przykładów uczących na raz. Sieć rekurencyjna, inicjalizowana potencjalnie różnymi `h0` dla różnych przykładów uczących, na raz przetwarza pierwsze elementy każdej sekwencji. Następnie przetwarza wszystkie drugie elementy, wszystkie trzecie elementy itd. Przetwarzanie odbywać się będzie po kolei w ramach kolejnych wierszy domyślnej repezentacji `batch_first=False` tj. równocześnie przetwarzane będą wszystkie sekwencje w batchu.

Implementacja dynamicznego tworzenia paczek danych (wraz z uzupełnianiem `<PAD>`) jest możliwa dzięki przekazaniu do konstruktora `DataLoader` funkcji `collate_fn`. Funkcja ta na wejście otrzymuje kolekcję danych (w takiej postaci w jakiej zostały one zwrócone z `DataSet` a na wyjście powinna zwrócić tensory reprezentujące paczkę danych. Nasza funkcja `collate_fn` powinna zwrócić 3 tensory. Pierwszy tensor powinien zawierać przykłady uczące (o równej długości), drugi odpowiednio uzupełnione sekwencje tagów oraz trzeci tensor jednowymiarowy (wektor) zawierający długości kolejnych sekwencji w batchu.

*UWAGA*: Konwencją umożliwiającą potem efektywniejsze przetwarzanie jest to, że sekwencje w batchu są posortowane ich długościami. Pierwsza sekwencja powinna być najdłuższa, a ostatnia najkrótsza.

In [67]:
def collate_fn(batch):
    batch = sorted(batch, key=lambda x: len(x['text']), reverse=True)
    txts, tags, sizes = [], [], []
    for i in batch:
        txts.append(torch.tensor(i['text']))
        tags.append(torch.tensor(i['label']))
        sizes.append(len(i['text']))
    return pad_sequence(txts, padding_value=0, batch_first=True), \
            pad_sequence(tags, padding_value=0, batch_first=True), \
            torch.tensor(sizes)


In [18]:
dataloader = DataLoader(dataset, batch_size=4, collate_fn=collate_fn)
for i in dataloader:
    print(i)
    break

(tensor([[   1,    9,    3, 1775,   17, 1164,  177,    6,  212,  317, 1216, 6116,
          108,  650,  597,   11,  457,    2,  249,   11, 2626, 2963,    2, 4920,
            6,  602,    7, 1401, 1752,   21,  732,    8,  543,   10, 8902,    1,
            4],
        [ 253,   36,  229,    3,  496,    5,    3,  118,  214, 1190,    6,  197,
          977,  172,    6, 1117,   19,   21,   49,  312,  224,   19,  417,    6,
         4096,    3, 1775,    2, 1143,    8,  384,  339,   56,  246,  138,    4,
            0],
        [  55,  246,    1, 2085,  477,   11, 2950,   36,   63, 6154,   25,    3,
         5984,   10, 1048,    6, 2788,  113,   69,  260, 2043,    9,   64, 4593,
          225, 3267,   88,  817,    4,    0,    0,    0,    0,    0,    0,    0,
            0],
        [2263,    5,    3, 4530, 4614, 1704,   10, 4293, 2543,    6,    7,  228,
         1314,  260,   36,  794,    6, 1916,    7,    1,    9, 2950,   96,    3,
          230,  126,    4,    0,    0,    0,    0,    0,    

Uzupełnienie sekwencji tokenami `<PAD>` do pełnej długości wydaje sie być dobrym pomysłem i pozwala na operowanie na tensorach które łatwo przesłać na kartę graficzną czy wykonać równoległe obliczenia na wszystkich jego elementach. Niemniej jednak fakt, że nasze sekwencje nie są równiej długości jest nadal niezwykle istotny przy wielu różnych obliczeniach. W szczególności jest on istotny przy przetwarzaniu sekwencji przez sieć rekurencyjną. W przypadku sieci rekurencyjnej przetwarzającej sekwencję uzupełnioną do końca zerami, sieć nie zwróci reprezentacji dla każdego słowa ale dla wszystkich elementów sekwencji czyli także dla końcowych elementów-zer. Ewidentnie sieć wykonała wiele niepotrzebnych operacji obliczeniowych tracąc czas (a przecież sieci rekurencyjne do najszybszych nie należą) jednakże prawidłowy wynik nadal jest do odzyskania. Możemy przecież post-factum wziąć pod uwagę tylko (długość sekwencji)-pierwszych elementów wyniku, uzyskując taki sam wynik jak przy przetwarzaniu sekwencji bez uzupełnienia.

Tak się jednak nie dzieje, gdy rozważamy dwukierunkową sieć rekurencyjną. Sieć iterująca w tył rozpoczęła przetwarzanie od początkowego stanu ukrytego, a następnie akutalizowała go za kolejne wejścia uważając sekwencję tokenów `<PAD>`! Wynik obliczeń tej sieci jest więc różny niż dla sekwencji nieuzupełnionej i co więcej nie jest on do odzyskania. Uzupełniając sekwnecję np. zerami nie tylko wykonujemy nadmiarowe obliczenia, ale także zaburzamy wyniki.

Na szczęście sieci rekurencyjne zaimplementowane w PyTorch obsługją także specjalny format danych tzw. `PackedSequence`, który przechowuje informację o długościach sekwencji zapewniając oszczędność obliczeń i takie same wyniki jak dla sekwencji bez uzupełniania. Przed wykonaniem obliczeń siecią rekurencyjną możemy dane zapakować do tego formatu, a następnie przetworzony wynik możemy z powrotem odpakować do postaci uzupełnionego do pełnej długości tensora. Pomocne są w tym dwie funkcje: `pack_padded_sequence` pakująca dane do tego formatu oraz funkcja odpakowująca `pad_packed_sequence`.

In [14]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

data = torch.tensor([[1,2,3,4,5],[-1,-2,-3,0,0], [10,20,0,0,0]])
lengths = torch.tensor([5,3,2])

packed_data = pack_padded_sequence(data, lengths, batch_first=True)
print(packed_data)

PackedSequence(data=tensor([ 1, -1, 10,  2, -2, 20,  3, -3,  4,  5]), batch_sizes=tensor([3, 3, 2, 1, 1]), sorted_indices=None, unsorted_indices=None)


Zapakowaliśmy 3 sekwencje, które uzupełnione były do długości 5. Ponieważ kolejne sekwencje są umieszczone w kolejnych wierszach tensora należy wyspecyfikować argument `batch_first=True`. Format `PackedSequence` przechowuje tensor `data` oraz tensor `batch_sizes`. Tak jak opisywaliśmy, sieć rekurencyjna zrównolegla swoje działania poprzez jednoczene przetwarzanie wszystkich sekwencji na raz, iterując po ich kolejnych elementach. Z tego powodu `batch_sizes` opisują wielkość przetwarzanej paczki danych (tj. liczba przetwarzanych sekwencji) w każdej iteracji. Na początku przetwarzamy 3 sekwencje, potem znowu 3, a następnie tylko dwie, gdyż ostatnia sekwencja miała tylko 2 elementy. Prześledź, że właśnie w taki sposób zostały ułożone dane w `data`.

Porównajmy działanie neuronu rekurencyjnego dla danych uzupełnionych tokenami `<PAD>` oraz dla danych spakowanych do `PackedSequence`.

In [20]:
N_FEATURES = 1
N_LAYERS = 1
HIDDEN_SIZE = 1

data = torch.FloatTensor([[1,2,3,4,5],[-1,-2,-3,0,0], [10,20,0,0,0]]).view(3,5,1)
lengths = torch.tensor([5,3,2])

rnn = nn.RNN(N_FEATURES, HIDDEN_SIZE, N_LAYERS, batch_first = True) 
out, _ = rnn(data)
print("Wyjście sieci dla danych o stałej długości: ", out)

packed_data = pack_padded_sequence(data, lengths, batch_first=True)
out_packed, _ = rnn(packed_data)
print("Wyjście sieci dla danych spakowanych: ",out_packed)
out, out_len = pad_packed_sequence(out_packed, batch_first=True)
print("Wyjście sieci po rozpakowaniu: ", out)


Wyjście sieci dla danych o stałej długości:  tensor([[[-0.6768],
         [-0.7637],
         [-0.9402],
         [-0.9834],
         [-0.9965]],

        [[ 0.6695],
         [ 0.7610],
         [ 0.9390],
         [-0.7100],
         [ 0.5777]],

        [[-1.0000],
         [-1.0000],
         [ 0.7310],
         [-0.5993],
         [ 0.5044]]], grad_fn=<TransposeBackward1>)
Wyjście sieci dla danych spakowanych:  PackedSequence(data=tensor([[-0.6768],
        [ 0.6695],
        [-1.0000],
        [-0.7637],
        [ 0.7610],
        [-1.0000],
        [-0.9402],
        [ 0.9390],
        [-0.9834],
        [-0.9965]], grad_fn=<CatBackward0>), batch_sizes=tensor([3, 3, 2, 1, 1]), sorted_indices=None, unsorted_indices=None)
Wyjście sieci po rozpakowaniu:  tensor([[[-0.6768],
         [-0.7637],
         [-0.9402],
         [-0.9834],
         [-0.9965]],

        [[ 0.6695],
         [ 0.7610],
         [ 0.9390],
         [ 0.0000],
         [ 0.0000]],

        [[-1.0000],
       

Zwróć uwagę jak ułożone są spakowane wyniki sieci.

Stworzyliśmy zbiór danych, mamy także zaimplementowany mechanizm dynamicznych paczek danych, a także wiemy jak efektywnie wykorzystywać je do obliczeń siecami rekurencyjnymi. Połóżmy więc wisienkę na torcie i stwórzmy model do tagowania tych danych. Model powinien składać się z warstwy zanurzeń słów, która jest wejściem do jednowarstwowego, dwukierunkowego LSTM. Ostatecznie predykcja jest wykonywana przez warstwę liniową (softmax).

In [11]:
WORD_EMBEDDING = 20

class TaggerNet(nn.Module):
    def __init__(self, vocab_size, hidden_size, n_tags):
        """ Argumentami konstruktora jest rozmiar słownika, 
            liczba neuronów w warstwie rekurencyjnej 
            oraz liczba klas/tagów
        """
        super(TaggerNet, self).__init__()
        self.embedding = nn.Embedding(vocab_size, WORD_EMBEDDING)
        self.lstm = nn.LSTM(WORD_EMBEDDING, hidden_size, batch_first=True, bidirectional=True)
        self.linear = nn.Linear(hidden_size * 2, n_tags)
        self.n_tags = n_tags
        
    def forward(self, sentence, seq_lengths):
        """ Wejściem jest paczka zdań (słowa reprezentowane przez indeksy)
            oraz tensor ich długości 
        """
        embedded_words = self.embedding(sentence)
        packed_words = pack_padded_sequence(embedded_words, seq_lengths, batch_first=True)
        lstm_out, _ = self.lstm(packed_words)
        unpacked_out, _ = pad_packed_sequence(lstm_out, batch_first=True)
        out = self.linear(unpacked_out)
        return out.view(len(sentence), self.n_tags, -1)
        



Zaimplemetuj finalną pętlę uczącą model. 
- Powinieneś wykorzystać błąd entropii krzyżowej jako funkcję straty policzoną dla każdego przewidzianego taga. 
- Zwróć uwagę, że wyniki sieci są w postaci rozpakowanej tj. uzupełnionej zerami do pełnej długości. Błąd nie powinien być liczony dla tych predykcji! Możesz to osiągnąć np. sprytnie wykorzystując parametr `ignore_index` klasy `nn.CrossEntropyLoss`. 
- Jako algorytm optymalizacyjny wykorzystaj `torch.optim.Adam`.
- Jedną z technik stabilizujących trening sieci jest przycinanie gradientu. W PyTorch możesz to uzyskać wywołując pomiędzy obliczeniem gradientu a wywołaniem kroku optymalizatora funkcji `nn.utils.clip_grad_norm_(model.parameters(), TRESHOLD)`. Zwróć uwagę, że nazwa funkcji zakończona jest `_` to znaczy, że operacja jest wykonywana in-place i wyniku funkcji nie trzeba podstawiać do żadnej zmiennej.
- Celem ćwiczenia nie jest wybór hiperparametrów, ani uzyskiwanie wysokich trafności.

In [25]:
model = TaggerNet(dataset.get_vocab_size(), 10, dataset.get_tagset_size())
dataloader = DataLoader(dataset, batch_size=20, collate_fn=collate_fn)

loss_function = nn.CrossEntropyLoss(ignore_index=0)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(5):  
    loss_sum = torch.tensor(0.)
    for sentences, tags, lengths in dataloader:
        
        optimizer.zero_grad()
        preds = model(sentences, lengths)
        loss = loss_function(preds, tags)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 2)
        optimizer.step()
        with torch.no_grad():
            loss_sum += loss
    print("iter %r: avg. train loss=%.4f" % (epoch+1, loss_sum/len(dataloader)))



iter 1: avg. train loss=2.0744
iter 2: avg. train loss=1.9407
iter 3: avg. train loss=1.9137
iter 4: avg. train loss=1.8927
iter 5: avg. train loss=1.8781


**Ćwiczenia**
- W tym zadaniu zaimplementowaliśmy dynamiczne tworzenie paczek danych, w ramach których uzupełnialiśmy sekwencje do takiej samej długości. Czasami stosowaną techniką jest posortowanie zbioru danych, tak aby sekwencje były ułożone od najdłuższej do najkrótszej, a następnie dynamicznie tworzy się kolejne paczeki danych iterując po tak ułożonym zbiorze. Jakie są wady i zalety takiego rozwiązania?
- Student podczas treningu sieci neuronowej zauważył, że wyniki funkcji celu w czasie optymalizacji modelu nie są stabilne. Spodziewa się, że zwiększenie rozmiaru paczki danych przyczyni się do bardziej stabilnego treningu sieci i poprawi wyniki. Jednakże, dalsze zwiększenie paczki danych nie jest możliwe gdyż nie mieści się ona w pamięci na karcie GPU. Sama implementacja sieci i reprezentacji zbioru danych jest wg. studenta optymalna, inna karta GPU o większej pamięci nie jest dostępna. Jak rozwiązac ten problem? Podaj zarys implementacji. 


Odpowiedź na ostatnią kropkę umieść poniżej.


Student może rozwiązać ten problem korzystając z techniki akumulacji gradientu. Polega ona na gromadzeniu, kumulacji wartości gradientów przez określoną liczbę - N - minibatchy przed wykonaniem jednego kroku optymalizacji i wyzerowania wartości gradientów. Pozwala ona dzięki temu na imitację większego rozmiaru docelowego batcha i rozwiązanie ograniczeń sprzętowych. Należy pamiętać przy implementacji o odpowiedniej normalizacji funkcji straty. Zarys implementacji:
```
accum_iter = 4
# minibatch_size = 8 => batch_size minibatch_size * accum_iter = 32 
for batch_idx, (inputs, labels) in enumerate(dataloader):
    preds = model(inputs)
    loss = loss_function(preds, labels)

    loss = loss / accum_iter
    loss.backward()
    if ((batch_idx + 1) % accum_iter == 0) or (batch_idx + 1 == len(dataloader)):
        optimizer.step()
        optimizer.zero_grad()
```

## Zadanie 3
Celem ostatniego ćwiczenia jest rozszerzenie modelu o warstwę CRF (nie można korzystać z gotowców). Implementacja:
- powinna operować tylko na jednej sekwencji (brak obsługi paczek danych w programowaniu dynamicznym)
- powinna być stabilna numerycznie tj. operować na logarytmach prawdopodobieństw (patrz rozdział "Uwaga implementacyjna" w wykładzie o CRF)
- warstwa CRF powinna być zaimplementowana wg. schematu "rozbicie funkcji oceny na ocenę emisji i tranzycji" $$\Psi_i (y_i, y_{i-1}, \vec{x}) = \exp\left({W}_{y_i}h_i\right) \exp\left(\vec{b}_{y_i, y_{i-1}} \right)$$
 gdzie ${W}_{y_i}h_i$ to ocena kompatybilności emisji, a $\vec{b}_{y_i, y_{i-1}}$ to kompatybilność tranzycji. Reprezentacja $h_i$ jest obliczana poprzez sieć rekurenycjną LSTM. Zwróć więc uwagę, że ${W}_{y_i}h_i$ to wynik warstwy liniowej mającej tyle neuronów wyjściowych ile tagów, za wejście przyjmując stan ukryty sieci rekurencyjnej. Z kolei $\vec{b}_{y_i, y_{i-1}}$ to jedna waga reprezentująca ocenę tranzycji, która jest dodatkowym parametrem warstwy CRF. Potrzebujemy więc macierz takich wag o wymiarach $C\times C$ gdzie C to liczba tagów. 
- Model powinien uwzględniać też reprezentacje tagów `START` i `STOP`. W rezultacie więc wygodnie zaimplementować macierz parametrów oceniających tranzycję jako macierz o wymiarach $C+2\times C+2$.
 
W czasie treningu modelu powinieneś maksymalizować logarytmiczną funkcję wiarygodności, co przy paczce danych o wielkości 1 sprowadza się do maksymalizacji logarytmu prawdopodobieństwa prawidłowej sekwencji. 
$$\max \log P({y} | {x}) = \log \frac{\prod_{i=1}^n \Psi_i (y_i, y_{i-1}, {x})}{Z({x})}  = \sum_{i=1}^n  \log \Psi_i (y_i, y_{i-1}, {x}) - \log Z({x})  $$
W PyTorch algorytmy optymalizacyjne minimalizują funkcje celu, więc wynik przed policzeniem gradientów musisz pomnożyć przez $-1$.

Do dyspozycji masz funkcję `log_sum_exp`:

In [8]:
def log_sum_exp(vec):
    max_score = torch.max(vec)
    return max_score + torch.log(torch.sum(torch.exp(vec - max_score)))

Uzupełnij klasę `TaggerCRFNet` takby aby implementowała powyższą architekturę.

In [113]:
class TaggerCRFNet(nn.Module):
    def __init__(self, vocab_size, hidden_size, n_tags):
        super(TaggerCRFNet, self).__init__()
        self.embedding = nn.Embedding(vocab_size, WORD_EMBEDDING)
        self.lstm = nn.LSTM(WORD_EMBEDDING, hidden_size, bidirectional = True)
        self.fc = nn.Linear(hidden_size * 2, n_tags)
        self.n_tags = n_tags
        self.START_TAG = self.n_tags 
        self.STOP_TAG = self.n_tags + 1
        self.transitions = nn.Parameter(torch.randn((n_tags + 2, n_tags + 2)))


    def _get_features(self, sentence, seq_lengths):
        """
        Funkcja ekstrakcji cech przez sieć rekurencyjną LSTM.
        Wyjście sieci zwraca od razu ocenę emisji każdego taga W_{y_i}*h_i
        
        Zwróć uwagę na wywołanie .view() na wyniku funkcji. Wynik ma 
        wymiarowość długość sekwencji x liczba tagów (brak obsługi kilku sekwencji na raz)
        """
        embedded_words = self.embedding(sentence)   
        packed_words = pack_padded_sequence(embedded_words, seq_lengths)
        out, _ = self.lstm(packed_words)                 
        unpacked, unpacked_len = pad_packed_sequence(out)
        out = self.fc(unpacked)         
        return out.view(-1, self.n_tags)  
    
    def _get_numerator(self, features, tags):
        """
        Funkcja przyjmuje na wejście features zwracane przez _get_features
        oraz sekwencję tagów. Obliczany jest (zlogarytmowany) licznik prawdopodobieństwa 
        podanej sekwencji. (Pierwszy term podanej wyżej funkcji celu)
        
        Zwróć uwagę w jaki sposób jest wykorzystywana macierz transitions.
        Pierwszym indeksem jest *kolejny* tag
        """
        score = torch.zeros(1)
        # Funkcje kompatybilności emisji
        for i, tag in enumerate(tags):
            score = score + features[i,tag]
        # Funkcje kompatybilności tranzycji
        score = score + self.transitions[tags[0], self.START_TAG]
        for i in range(len(tags)-1):
            score = score + self.transitions[tags[i+1], tags[i]]
        score = score + self.transitions[self.STOP_TAG,tags[-1]]
        return score
    
    def _forward_alg(self, features, seq_lengths):
        """
        Funkcja zwracająca logarytm sumy statystycznej (drugi term funkcji celu)
        """
        alphas = features[0] + self.transitions[:self.n_tags , self.START_TAG].view(-1)
        for feature in features[1:]:
            z = alphas + feature.unsqueeze(1) + self.transitions[:self.n_tags, :self.n_tags]
            z_i = z.max(1).values
            alphas = z_i + (z - z_i.unsqueeze(1)).logsumexp(1)
        
        return log_sum_exp(alphas + self.transitions[self.STOP_TAG, :self.n_tags])

    def seq_log_probability(self, sentence, tags, seq_lengths):
        """
        Funkcja zwracająca logarytm prawdopodobieństwa podanej sekwencji
        """
        features = self._get_features(sentence, seq_lengths)
        numerator = self._get_numerator(features, tags)
        partition_function = self._forward_alg(features,seq_lengths)
        return numerator - partition_function

    def forward(self, sentence, seq_lengths):
        """
        Funkcja zwracająca najbardziej prawdopodobną sekwencję tagów dla podanego wejścia.
        
        Porada: zaimplementuj ją w drugiej kolejności, jak seq_log_probability będzie prawidłowo działać
        """
        features = self._get_features(sentence, seq_lengths)
        alphas = features[0] + self.transitions[:self.n_tags, self.START_TAG].view(-1)
        pred_tags = []

        for feature in features[1:]:
            z = alphas + feature.unsqueeze(1) + self.transitions[:self.n_tags, :self.n_tags]
            preds, tags = z.max(1)
            pred_tags.append(tags)
            alphas = preds + (z - preds.unsqueeze(1)).logsumexp(1)
        
        alphas += self.transitions[self.STOP_TAG, :self.n_tags]
        
        tag = int(alphas.max(0).indices)
        result = [tag]

        for next_tag in reversed(pred_tags):
            tag = int(next_tag[tag])
            result.append(tag)

        result.reverse()
        return result
        

    def score_seq(self, sentence, tags, seq_lengths):
        """
        Funkcja dla testów implementacji.
        Wyznacz ocenę sekwencji (zlogarytmowany licznik). 
        """
        features = self._get_features(sentence, seq_lengths)
        numerator = self._get_numerator(features, tags)
        return numerator 
    
    def partition_function(self, sentence, tags, seq_lengths):
        """
        Funkcja dla testów implementacji.
        Wyznacz sumę statystyczną (zlogarytmowany mianownik). 
        """
        features = self._get_features(sentence, seq_lengths)
        gold_score = self._forward_alg(features, seq_lengths)
        return  gold_score 
    


Poprawność algorytmu "w przód i w tył" mozna prosto sprawdzić, porównując wynik z wynikiem algorytmu zachłannego (tj. iterującym po wszystkich sekwencjach), dla krótkiej sekwencji z małą liczbą możliwych tagów.

In [100]:
import itertools

data = torch.tensor([1, 2, 5, 10, 11, 3]).view(6,1)
tags = torch.tensor([1, 2, 0, 1, 1, 1]).view(6,1)
length = torch.tensor([6])

model = TaggerCRFNet(vocab_size=12, hidden_size=3, n_tags=3)
sum_forward_alg = model.partition_function(data,tags, length)
print('Programowanie dynamiczne',sum_forward_alg)
sum_score= []
for i in itertools.product(range(3),repeat=6):
    new_tags = torch.tensor(i).view(6,1)
    sum_score.append(model.score_seq(data, new_tags, length))
print("Algorytm zachłanny",log_sum_exp(torch.tensor(sum_score)))

Programowanie dynamiczne tensor(8.7825, grad_fn=<AddBackward0>)
Algorytm zachłanny tensor(8.7825)


Ostatecznie zaimplementuj trening modelu.

In [114]:
CLIP_TRESHOLD = 5. 

model = TaggerCRFNet(dataset.get_vocab_size(), 10, dataset.get_tagset_size())
dataloader = DataLoader(dataset, batch_size=1, collate_fn=collate_fn )
optimizer = torch.optim.Adam(model.parameters())

for epoch in range(1):  # dla testów wystarczy jedna epoka, możesz przerwać obliczenia w jej trakcie
    loss_sum = torch.tensor([0.])
    for i, (sentences, tags, lengths) in enumerate(dataloader):
        loss = -model.seq_log_probability(sentences.T, tags.T, lengths)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), CLIP_TRESHOLD)
        optimizer.step()
        optimizer.zero_grad()
        with torch.no_grad():
            loss_sum += loss
            if i% 10 == 9:
                print("Loss function: ",loss_sum/10)
                # Wyniki nadal będą niestabilne, ale ogólna tendencja liczb powinna być malejąca
                loss_sum = 0



Loss function:  tensor([114.8116])
Loss function:  tensor([84.8760])
Loss function:  tensor([91.0444])
Loss function:  tensor([79.3839])
Loss function:  tensor([60.6852])
Loss function:  tensor([96.4784])
Loss function:  tensor([75.5877])
Loss function:  tensor([76.8009])
Loss function:  tensor([77.3414])
Loss function:  tensor([62.9236])
Loss function:  tensor([59.7441])
Loss function:  tensor([50.8479])
Loss function:  tensor([57.7033])
Loss function:  tensor([58.6288])
Loss function:  tensor([49.8769])
Loss function:  tensor([61.5979])
Loss function:  tensor([26.1501])
Loss function:  tensor([54.5282])
Loss function:  tensor([42.2402])
Loss function:  tensor([41.4243])
Loss function:  tensor([63.1894])
Loss function:  tensor([61.5643])
Loss function:  tensor([59.7843])
Loss function:  tensor([41.3084])
Loss function:  tensor([34.0270])
Loss function:  tensor([34.7283])
Loss function:  tensor([40.2161])
Loss function:  tensor([50.3928])
Loss function:  tensor([46.3197])
Loss function

KeyboardInterrupt: 

In [115]:
import numpy as np

for i, (sentences, tags, lengths) in enumerate(dataloader):
    preds = model(sentences.T, lengths)
    Y, y = np.array(preds), tags[0].numpy()
    print('Batch', i)
    print('Labels:', tags[0].tolist())
    print('Predicted:', preds)
    print(f'accu = {(Y == y).mean()*100:.4f}%\n')
    if i == 10: break

Batch 0
Labels: [2, 5, 2, 1, 4, 6, 6, 6, 6, 2, 1, 1, 8, 2, 1, 5, 2, 3, 9, 5, 2, 2, 3, 4, 6, 6, 2, 1, 1, 5, 2, 1, 1, 2, 1, 1, 3]
Predicted: [2, 5, 2, 1, 4, 2, 1, 5, 2, 1, 4, 2, 1, 1, 1, 5, 2, 3, 2, 5, 2, 1, 3, 2, 5, 2, 2, 1, 1, 5, 2, 1, 1, 2, 1, 1, 3]
accu = 62.1622%

Batch 1
Labels: [3, 5, 2, 1, 2, 1, 2, 1, 1, 5, 2, 1, 1, 1, 4, 6, 6, 6, 2, 1, 5, 2, 5, 2, 1, 1, 3]
Predicted: [2, 5, 2, 1, 1, 1, 2, 1, 5, 5, 2, 1, 1, 1, 1, 1, 5, 4, 2, 1, 5, 2, 1, 2, 1, 1, 3]
accu = 70.3704%

Batch 2
Labels: [3, 2, 4, 2, 1, 5, 2, 4, 6, 6, 5, 2, 1, 2, 1, 4, 6, 2, 1, 1, 1, 5, 2, 1, 1, 1, 2, 1, 3]
Predicted: [2, 4, 2, 1, 1, 5, 2, 1, 1, 1, 5, 2, 1, 2, 1, 5, 4, 2, 1, 1, 1, 5, 2, 1, 1, 1, 2, 1, 3]
accu = 68.9655%

Batch 3
Labels: [2, 4, 6, 2, 1, 5, 2, 1, 4, 6, 6, 6, 2, 1, 5, 2, 1, 5, 2, 1, 1, 1, 1, 4, 6, 2, 1, 3, 2, 3, 2, 1, 1, 1, 4, 3]
Predicted: [2, 1, 4, 2, 1, 5, 2, 1, 1, 1, 5, 2, 1, 1, 5, 2, 1, 5, 2, 1, 1, 1, 1, 5, 4, 2, 1, 3, 2, 3, 2, 1, 1, 1, 1, 3]
accu = 72.2222%

Batch 4
Labels: [3, 2, 1, 5, 2, 5, 2, 1, 1